In [6]:
# Import necessary libraries
import csv
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from sklearn.model_selection import train_test_split


# Sample data representing air pressure readings for each drone
# Each row represents a drone, and each column represents a feature (e.g., microphone array readings)
data_Altura = []
data_ANWB = []
data_ATMOS = []
data_Autel_Evo = []
data_Phantom = []
name_dic = {'Altura.csv': data_Altura, 'ANWB.csv': data_ANWB, 'ATMOS.csv': data_ATMOS, 'Autel_Evo.csv': data_Autel_Evo, 'Phantom.csv': data_Phantom}

def datasets():
    data_lst = ['Altura.csv', 'ANWB.csv', 'ATMOS.csv', 'Autel_Evo.csv', 'Phantom.csv']
    for filename in data_lst:
        data = []
        with open(filename, "r") as file:
            csvreader = csv.reader(file)
            for row in csvreader:
                row1 = []
                for i,e in enumerate(row):
                    row1.append(float(e))
                data.append(row1)
        step = np.array(data[:249999])
        for i,row in enumerate(step.T):
            if i != 0 and i != 16 and  i !=20 and i != 40 and  i !=62 and  i !=63:
                name_dic[filename].append(row)

datasets()

In [7]:
#Defining SVM classifier (extracted form AI course)

# Set the PyTorch and numpy random seeds for reproducibility:
seed = 0
torch.manual_seed(seed)
np.random.seed(seed)


class MLPClassifier(torch.nn.Module):

    def __init__(self, n_features, n_hidden_neurons, n_classes, learning_rate, n_epochs):
        """
        Initialize the neural network classifier 
        """
        # initialize superclass
        super(MLPClassifier,self).__init__()

        # the number of classes
        self.n_classes = n_classes

        # the number of epochs
        self.n_epochs = n_epochs
        
        # create the neural network
        self.network = torch.nn.Sequential(torch.nn.Linear(n_features, n_hidden_neurons),
                            torch.nn.ReLU(),torch.nn.Linear(n_hidden_neurons,n_hidden_neurons),torch.nn.ReLU(),torch.nn.Linear(n_hidden_neurons,n_classes))



        # the cross-entropy loss function
        self.loss_fn = torch.nn.CrossEntropyLoss()

        # the Adam optimizer
        self.optimizer = torch.optim.Adam(self.network.parameters(), lr=learning_rate)


    def forward(self, X):
        """
        Forward pass of the neural network
        """
        return self.network(X)


    def train(self, X_train, y_train):
        """
        Train the neural network classifier 
        """

        # convert data into appropriate format for {torch}
        X_train_torch = torch.tensor(X_train, dtype=torch.float32)
        y_train_torch = torch.nn.functional.one_hot(torch.tensor(y_train), num_classes=self.n_classes).to(dtype=torch.float32)

        # start training
        print("Training started...")
        for epoch in range(0, self.n_epochs):
            # Set optimizer to zero grad
            self.optimizer.zero_grad()

            # Forward step
            y_pred = self.forward(X_train_torch)

            # Compute loss
            loss = self.loss_fn(y_pred, y_train_torch)

            # Backward step
            loss.backward()

            # Update weights
            self.optimizer.step()

            # Report loss
            if not ((epoch+1) % 100):
                print("Epoch:", epoch+1, f", loss = {loss.detach().item():.4f} ")
        print(f"Training completed in {epoch+1} epochs! Final loss = {loss.detach().item():.4f}")


    def predict(self, X_test):
        """
        Use the trained neural network to predict the labels of the test set 
        """
        # convert data into appropriate format for {torch}
        X_test_torch = torch.tensor(X_test, dtype=torch.float32)
        arg = self.forward(X_test_torch)
        return torch.argmax(arg,dim=1)


In [16]:
# Slice the data into a specified number (percentage - sample_percent) of separate datasets
# e.g. 4 training datasets and 1 test data set, or 3 training data sets and 2 test data sets
def data_clean(drone, sample_percent):
    X_data = []
    data_arr = np.array(name_dic[drone]).T
    total_time_points_no = len(data_arr)
    sample_size = int(total_time_points_no * sample_percent)
    for i in range(0,int(1/sample_percent)):
        X = data_arr[sample_size*i:sample_size*(i+1)].T
        X_data  += [X]
    return np.array(X_data)

sample_percent = 0.01
X_data_Altura = data_clean('Altura.csv', sample_percent)
X_data_ANWB = data_clean('ANWB.csv', sample_percent)
X_data_ATMOS = data_clean('ATMOS.csv', sample_percent)
X_data_Autel_Evo = data_clean('Autel_Evo.csv', sample_percent)
X_data_Phantom = data_clean('Phantom.csv', sample_percent)

# Split the data into training and testing sets
X_data = []
y_data = []
for i in X_data_Altura:
    for e in i:
        X_data += [e]
        y_data += [0]
for i in X_data_ANWB:
    for e in i:
        X_data += [e]
        y_data += [1]
for i in X_data_ATMOS:
    for e in i:
        X_data += [e]
        y_data += [2]
for i in X_data_Autel_Evo:
    for e in i:
        X_data += [e]
        y_data += [3]
for i in X_data_Phantom:
    for e in i:
        X_data += [e]
        y_data += [4]
train, test, train_labels, test_labels = train_test_split(X_data, y_data, test_size=0.20, random_state=42)

# Create an SVM classifier
n_features = 2499
n_hidden_neurons = 50 #120 
n_classes = 5
learning_rate = 0.02 #0.008
n_epochs = 1000 #3000
clf = MLPClassifier(n_features, n_hidden_neurons, n_classes, learning_rate, n_epochs)

# Train the classifier
clf.train(train,train_labels)

# Make predictions on the test set
y_predict = clf.predict(test)

# Calculate accuracy
accuracy = accuracy_score(test_labels, y_predict, normalize=True)

print("Prediction accuracy:", accuracy)

Training started...
Epoch: 100 , loss = 0.0486 
Epoch: 200 , loss = 0.0105 
Epoch: 300 , loss = 0.0042 
Epoch: 400 , loss = 0.0018 
Epoch: 500 , loss = 0.0008 
Epoch: 600 , loss = 0.0004 
Epoch: 700 , loss = 0.0003 
Epoch: 800 , loss = 0.0002 
Epoch: 900 , loss = 0.0001 
Epoch: 1000 , loss = 0.0001 
Training completed in 1000 epochs! Final loss = 0.0001
tensor([4, 2, 3,  ..., 3, 2, 0])
Accuracy: 0.9360344827586207


In [17]:
#Input selected microphone recording to determine drone
test_data = clf.predict([X_data_Phantom[80][9]])
print(test_data) #make formatting/text that corresponds with the predicted drone

tensor([4])
